In [ ]:
#pip install langchain langchain_community chromadb langchain_google_genai bs4 numexpr

In [7]:
from langchain import LLMMathChain
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool, DuckDuckGoSearchRun
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain.chains import ConversationChain
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor
import os
import getpass

c:\Users\Diya\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
import os
import getpass
#enter your gemini api key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("API Key - ")

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=1)

In [11]:
class ConversationBuffer:
    def __init__(self):
        self.history = []

    def add_to_history(self, question, answer):
        self.history.append((question, answer))

    def get_history(self):
        return self.history
    def clear_history(self):
        self.history.clear()
        return self.history

def get_chat_history():
    history = conversation_buffer.get_history()
    #print("check")
    formatted_history = {"history": history}
    return formatted_history
conversation_buffer = ConversationBuffer()
print(conversation_buffer.history)

[]


In [12]:
def invoke_with_history(question):
    result = rag_chain.invoke(question)
    conversation_buffer.history[-1] = (question, result)
    return result

In [13]:
llm_prompt_template = """You are an assistant for question-answering tasks.
Take the question and find relevant information from the given context and return the generated answer.
Use the following context to answer the question.
Use chat history to get more context whenever necessary.
If you don't know the answer or if the answer is irrelevent, just say that you don't know as response.
Use five sentences maximum and keep the answer friendly.\n
Question: {question} \nContext: {context}\nChat History: {chatHistory} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

In [16]:
#give doc website link below or documents using doc loaders.
loader = WebBaseLoader("")
docs = loader.load()
text_content = docs[0].page_content
text_content_1 = text_content.split("code, audio, image and video.",1)[0]
final_text = text_content_1.split("Cloud TPU v5p",1)[0]
docs =  [Document(page_content=final_text, metadata={"source": "local"})]
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(
                     documents=docs,
                     embedding=gemini_embeddings,
                     persist_directory="./chroma_db"
                     )
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",
                        embedding_function=gemini_embeddings
                   )
retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 1})
print(len(retriever.get_relevant_documents("nissan")))

c:\Users\Diya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


1


In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough(),"chatHistory":lambda x:conversation_buffer.get_history()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [21]:
from langchain.agents import load_tools
#Enter the document topic in the tool description.
search_tool = Tool.from_function(
    func=rag_chain.invoke,
    name="SearchDoc",
    description="Useful when u need to find answers to the question related to the topic -  . pass user's input as is without any changes"
)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)
tools = [search_tool, math_tool]

c:\Users\Diya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm_math\base.py:58: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [22]:
from langchain.agents import AgentExecutor
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors = True
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

c:\Users\Diya\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [23]:
question="What is two plus two?"
conversation_buffer.add_to_history(question, None)
x=agent.invoke(question)["output"]
#print(llm.usage_metadeta)
conversation_buffer.history[-1] = (question,x )
print(x)



> Entering new AgentExecutor chain...
I should use Calculator to perform the addition
Action: Calculator
Action Input: 2+2

> Entering new LLMMathChain chain...
2+2```text
2+2
```
...numexpr.evaluate("2+2")...

Answer: 4
> Finished chain.

Observation: Answer: 4
Thought:I now know the final answer
Final Answer: 4

> Finished chain.
4


In [24]:
#to get full history of chat
print(conversation_buffer.get_history())

[('What is two plus two?', '4')]


In [ ]:
#to clear full history
print(conversation_buffer.clear_history())